<a href="https://colab.research.google.com/github/ara42/Blog_Project/blob/main/ml/Ad_or_Not.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 !pip install PyMySQL

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.3 MB/s eta 0:00:00


In [33]:
import pymysql
blogdb = pymysql.connect(host='blogdb.cm2yxwfja9ii.ap-northeast-2.rds.amazonaws.com',
                      user='admin',
                      password='blogdb!2',
                      database='blogdb',
                      charset='utf8',
                      port=3306)

cursor =blogdb.cursor(pymysql.cursors.DictCursor)

# 데이터불러오기
query = "SELECT id, ocr_text FROM post_data where map=1 And ad_status = 'NEED-TO-PROCESS' and res_id >29 and res_id<41"
cursor.execute(query)
results = cursor.fetchall()

In [27]:
results

{'id': 1,
 'ocr_text': 'NULL, RAMEN RESTAURANT OPEN CLOSE BREAK TIME LAST ORDER H'}

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# 데이터 불러오기
data = pd.read_csv('adocr.csv',encoding='euc-kr')
data = data.drop('Unnamed: 2', axis=1)

# 텍스트 데이터 토큰화 및 벡터화
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(data['content'])

# 훈련 데이터와 테스트 데이터 분리
x_train, x_test, y_train, y_test = train_test_split(x, data['label'], test_size=0.2, random_state=42)


# SVM 모델 생성 및 훈련
svm = SVC()
svm.fit(x_train, y_train)

# 모델 평가
y_pred = svm.predict(x_test)
print(classification_report(y_test, y_pred))

# 예측하고 싶은 텍스트 데이터 토큰화 및 벡터화
x_new = vectorizer.transform(['NO광고NO협찬'])

# 저장된 모델로 예측 수행
predicted_label =svm.predict(x_new)
print(predicted_label)

              precision    recall  f1-score   support

           0       0.96      0.99      0.97       145
           1       0.99      0.97      0.98       217

    accuracy                           0.98       362
   macro avg       0.98      0.98      0.98       362
weighted avg       0.98      0.98      0.98       362



In [24]:
# 예측하고 싶은 텍스트 데이터 토큰화 및 벡터화
x_new = vectorizer.transform([text])

# 저장된 모델로 예측 수행
predicted_label =svm.predict(x_new)
print(predicted_label)

[0]


In [34]:
for result in results:
    ocr_text = result['ocr_text']
    id = result['id']

    # 토큰화 및 벡터화
    x_new = vectorizer.transform([ocr_text])

    # SVM 모델로 예측
    predicted_label = svm.predict(x_new)

    # 예측 결과에 따라 ad_status 값을 업데이트
    if predicted_label == 0:
        ad_status = 'NON-AD'
    else:
        ad_status = 'AD'

    # ad_status 값을 업데이트하는 SQL 실행
    update_query = "UPDATE post_data SET ad_status = %s WHERE id = %s"
    cursor.execute(update_query, (ad_status, id))

blogdb.commit()


AttributeError: ignored